In [1]:
import os
# Change the working directory to strategy_lab
os.chdir("/Users/sugang/Desktop/projects/2024coin/strategy_lab")


import pandas as pd
import numpy as np
from handle_candle import resample_df
from datetime import datetime
from performance_utils import get_performance


In [2]:
def backtest(df, ma_short, ma_long):
    df = df.copy()
    # get ma values
    df[f"ma_{ma_short}"] = df["close"].rolling(window=ma_short).mean()
    df[f"ma_{ma_long}"] = df["close"].rolling(window=ma_long).mean()

    # Initialize the signal column with default value 0
    df["signal"] = 0

    # Generate buy signal: 1 when ma5 crosses above ma20
    df.loc[(df[f"ma_{ma_short}"] > df[f"ma_{ma_long}"]) & (df[f"ma_{ma_short}"].shift(1) <= df[f"ma_{ma_long}"].shift(1)), "signal"] = 1

    # Generate sell signal: -1 when ma_5 crosses below ma_20
    df.loc[(df[f"ma_{ma_short}"] < df[f"ma_{ma_long}"]) & (df[f"ma_{ma_short}"].shift(1) >= df[f"ma_{ma_long}"].shift(1)), "signal"] = -1


    # Initialize the position column with default value 0
    df["position"] = 0

    # Variable to track the current position status
    current_position = 0

    # Iterate over the rows and set the position
    for i in range(1, len(df)):
        if df.loc[i-1, "signal"] == 1:  # Enter long position
            if df.loc[i, "signal"] == -1:
                df.loc[i, "position"] = 1
                current_position =0
                continue
            current_position = 1
        elif df.loc[i, "signal"] == -1:  # Exit position
            df.loc[i, "position"] = current_position
            current_position = 0
            continue
        df.loc[i, "position"] = current_position

    # Calculate the strategy returns (only when in a long position)
    df["strategy_returns"] = df["position"] * df["close"].pct_change()
    df["strategy_returns2"] = df["strategy_returns"]

    # Adjust for trading fees (buy with 0.2% fee, sell with 0.2% fee)
    df["buy_price"] = df["close"].shift(1) * np.where(df["signal"].shift(1) == 1, 1.002, 1)
    df["sell_price"] = df["close"] * np.where(df["signal"] == -1, 0.998, 1)

    # Calculate strategy returns with fees
    df["strategy_returns2"] = np.where(df["position"] == 1, df["sell_price"] / df["buy_price"] - 1, 0)

    # Calculate the cumulative returns
    df["cumulative_returns"] = (1 + df["strategy_returns"]).cumprod()
    df["cumulative_returns2"] = (1 + df["strategy_returns2"]).cumprod()
    performance = get_performance(df)
    # print(f"ma{ma_short}, ma{ma_long}\n", performance)
    return {"strategy": f"ma{ma_short}_{ma_long}", "stoploss": None, **performance}

def backtest_stoploss(df, ma_short, ma_long, stop_loss_pct):
    df = df.copy()
    # get ma values
    df[f"ma_{ma_short}"] = df["close"].rolling(window=ma_short).mean()
    df[f"ma_{ma_long}"] = df["close"].rolling(window=ma_long).mean()

    # Initialize the signal column with default value 0
    df["signal"] = 0

    # Generate buy signal: 1 when ma5 crosses above ma20
    df.loc[(df[f"ma_{ma_short}"] > df[f"ma_{ma_long}"]) & (df[f"ma_{ma_short}"].shift(1) <= df[f"ma_{ma_long}"].shift(1)), "signal"] = 1

    # Generate sell signal: -1 when ma_5 crosses below ma_20
    df.loc[(df[f"ma_{ma_short}"] < df[f"ma_{ma_long}"]) & (df[f"ma_{ma_short}"].shift(1) >= df[f"ma_{ma_long}"].shift(1)), "signal"] = -1

    # Initialize the position column with default value 0
    df["position"] = 0
    df["highest_price"] = np.nan
    # Variable to track the current position status
    current_position = 0

    # Iterate over the rows and set the position
    for i in range(1, len(df)):
        if df.loc[i-1, "signal"] == 1:  # Enter long position
            df.loc[i, "highest_price"] = max(df.loc[i-1, "close"], df.loc[i, "close"])
            if df.loc[i, "signal"] == -1:
                df.loc[i, "position"] = 1
                current_position =0
                continue
            if df.loc[i, "close"] <= df.loc[i, "highest_price"] * (1 - stop_loss_pct):
                df.loc[i, "position"] = 1
                df.loc[i, "signal"] = -1
                current_position =0
                continue
            current_position = 1
        elif df.loc[i, "signal"] == -1 and current_position == 1:  # Exit position
            df.loc[i, "highest_price"] = max(df.loc[i-1, "highest_price"], df.loc[i, "close"])
            df.loc[i, "position"] = current_position
            current_position = 0
            continue

        elif current_position == 1:  # Check current_position instead of df position
            df.loc[i, "highest_price"] = max(df.loc[i-1, "highest_price"], df.loc[i, "close"])
            if df.loc[i, "close"] <= df.loc[i, "highest_price"] * (1 - stop_loss_pct):
                df.loc[i, "position"] = 1
                df.loc[i, "signal"] = -1
                current_position = 0
                continue
        
        if current_position == 0 and df.loc[i, f"ma_{ma_short}"] > df.loc[i, f"ma_{ma_long}"]:
            df.loc[i, "signal"] = 1
        df.loc[i, "position"] = current_position

    # Calculate the strategy returns (only when in a long position)
    df["strategy_returns"] = df["position"] * df["close"].pct_change()
    df["strategy_returns2"] = df["strategy_returns"]

    # Adjust for trading fees (buy with 0.2% fee, sell with 0.2% fee)
    df["buy_price"] = df["close"].shift(1) * np.where(df["signal"].shift(1) == 1, 1.002, 1)
    df["sell_price"] = df["close"] * np.where(df["signal"] == -1, 0.998, 1)

    # Calculate strategy returns with fees
    df["strategy_returns2"] = np.where(df["position"] == 1, df["sell_price"] / df["buy_price"] - 1, 0)

    # Calculate the cumulative returns
    df["cumulative_returns"] = (1 + df["strategy_returns"]).cumprod()
    df["cumulative_returns2"] = (1 + df["strategy_returns2"]).cumprod()
    performance = get_performance(df)
    # print(f"stoploss{stop_loss_pct}%, ma{ma_short}, ma{ma_long}\n", performance)
    df.to_csv(f"1_moving_averages/temp_stoploss_{ma_short}_{ma_long}_{stop_loss_pct}.csv")
    return {"strategy": f"ma{ma_short}_{ma_long}", "stoploss": stop_loss_pct, **performance}
    

def backtest_benchmark(df):
    df = df.copy()
    df["benchmark_returns2"] = df["close"].pct_change()
    df["cumulative_returns2"] = (1 + df["benchmark_returns2"]).cumprod()
    performance = get_performance(df)
    
    return {"strategy": "benchmark", "stoploss": None, **performance}

ma_shorts =[1, 2, 5]
ma_longs = [20, 50, 150, 200]
stop_loss_pcts = [0.03, 0.05, 0.1, 0.2]
hours = range(0, 24, 1)


# Store results in a list
results = []
# Read the data
df = pd.read_csv("temp.csv", index_col=0)

# Run backtests and collect results
results.append(backtest_benchmark(df))
for hour in hours:
    df_copy = df.copy()
    execution_time = datetime.strptime(f"{hour:02d}:00", "%H:%M")
    df_copy = resample_df(df_copy, execution_time=execution_time)
    for ma_short in ma_shorts:
        for ma_long in ma_longs:
            results.append({
                "execution_time": execution_time.strftime("%H:%M"),
                **backtest(df_copy, ma_short, ma_long)
            })
            for stop_loss_pct in stop_loss_pcts:
                results.append({
                    "execution_time": execution_time.strftime("%H:%M"),
                    **backtest_stoploss(df_copy, ma_short, ma_long, stop_loss_pct)
                })



KeyboardInterrupt: 

In [3]:
# Convert results to DataFrame and save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv("1_moving_averages/results.csv", index=False)
print("Results saved to results.csv")

Results saved to results.csv
